# Sistema de Ensemble Avanzado - Production Ready

Sistema de ensemble con arquitectura optimizada que integra los mejores algoritmos con ingenieria de features avanzada. Este notebook implementa un pipeline production-ready con mejor arquitectura que los enfoques tradicionales.

## Objetivos de Produccion
1. Sistema ensemble optimizado con arquitectura robusta
2. Ingenieria de features multi-tipo (numericas + TF-IDF avanzado)
3. Manejo inteligente de algoritmos sparse/dense
4. Sistema de calibracion y confianza
5. Pipeline completo para implementacion
6. Metricas y monitoreo comprehensivo

## Diferencias vs Notebook 08 (Tradicional)
- **Feature Engineering**: Extractor multi-tipo vs features basicas
- **Architecture**: Clase EnsembleBuilder vs funciones simples  
- **Error Handling**: Manejo robusto vs evaluacion directa
- **Algorithms**: Mejor seleccion y configuracion
- **Production Ready**: Sistema completo vs demo academico

Este sistema esta diseñado para superar los enfoques tradicionales mediante mejor arquitectura de datos y algoritmos optimizados.

In [1]:
# Configuracion de entorno y dependencias
import sys
import subprocess

# Detectar si estamos en Colab
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("Ejecutando en Google Colab")
    # Instalar dependencias para Colab
    !pip install plotly scikit-learn xgboost lightgbm
    # Montar Google Drive si es necesario
    from google.colab import drive
    # drive.mount('/content/drive')
else:
    print("Ejecutando en entorno local")
    # Verificar dependencias locales
    packages = ['plotly', 'scikit-learn', 'xgboost', 'lightgbm']
    for package in packages:
        try:
            __import__(package.replace('-', '_'))
        except ImportError:
            subprocess.check_call([sys.executable, "-m", "pip", "install", package])

print(f"Entorno configurado correctamente")



Ejecutando en entorno local
Entorno configurado correctamente


In [2]:
# Importacion de librerias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Machine learning y ensemble
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, f1_score, accuracy_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.calibration import CalibratedClassifierCV, calibration_curve

# Algoritmos base
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# Metodos de ensemble
from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier, VotingClassifier, StackingClassifier
import xgboost as xgb
import lightgbm as lgb

# Intentar importar CatBoost
try:
    import catboost as cb
    CATBOOST_AVAILABLE = True
    print("CatBoost importado correctamente")
except ImportError:
    CATBOOST_AVAILABLE = False
    print("CatBoost no disponible")

# Procesamiento de texto
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Utilidades
import os
import pickle
import joblib
from pathlib import Path
import time
from datetime import datetime

# Configuracion
np.random.seed(42)
if IN_COLAB:
    plt.style.use('default')
else:
    plt.rcParams['figure.figsize'] = (10, 6)

# Recursos NLTK
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)

print("Librerias importadas correctamente")
print(f"Timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

CatBoost importado correctamente
Librerias importadas correctamente
Timestamp: 2025-08-24 20:58:07


## Carga de Datos Truth Seeker

Cargo el dataset para el sistema de ensemble completo.

In [3]:
# Carga de datasets procesados
if IN_COLAB:
    # Para Colab - usar datos procesados
    df_features = pd.read_csv('dataset_features_processed.csv')
    df_text = pd.read_csv('text_data_for_nlp.csv')
else:
    # Para ejecucion local - usar datos procesados
    df_features = pd.read_csv('../processed_data/dataset_features_processed.csv')
    df_text = pd.read_csv('../processed_data/text_data_for_nlp.csv')

print(f"Dataset de features cargado: {df_features.shape[0]} filas y {df_features.shape[1]} columnas.")
print(f"Dataset de texto cargado: {df_text.shape[0]} filas y {df_text.shape[1]} columnas.")

print("\nColumnas features dataset:")
print(list(df_features.columns))

print("\nColumnas texto dataset:")
print(list(df_text.columns))

# Verificar columnas de interes
target_columns = [col for col in df_features.columns if 'target' in col.lower()]
text_columns = [col for col in df_text.columns if 'statement' in col.lower()]

print(f"\nColumnas target encontradas: {target_columns}")
print(f"Columnas texto encontradas: {text_columns}")

Dataset de features cargado: 134198 filas y 58 columnas.
Dataset de texto cargado: 134198 filas y 2 columnas.

Columnas features dataset:
['BinaryNumTarget', 'followers_count', 'friends_count', 'favourites_count', 'statuses_count', 'listed_count', 'BotScore', 'BotScoreBinary', 'cred', 'normalize_influence', 'mentions', 'quotes', 'replies', 'retweets', 'favourites', 'hashtags', 'URLs', 'unique_count', 'total_count', 'ORG_percentage', 'NORP_percentage', 'GPE_percentage', 'PERSON_percentage', 'MONEY_percentage', 'DATE_percentage', 'CARDINAL_percentage', 'PERCENT_percentage', 'ORDINAL_percentage', 'FAC_percentage', 'LAW_percentage', 'PRODUCT_percentage', 'EVENT_percentage', 'TIME_percentage', 'LOC_percentage', 'WORK_OF_ART_percentage', 'QUANTITY_percentage', 'LANGUAGE_percentage', 'Word count', 'Max word length', 'Min word length', 'Average word length', 'present_verbs', 'past_verbs', 'adjectives', 'adverbs', 'adpositions', 'pronouns', 'TOs', 'determiners', 'conjunctions', 'dots', 'exclama

In [4]:
# Preparacion y limpieza de datos
def prepare_ensemble_data():
    """Prepara datos combinando features numericas y texto para ensemble"""
    
    # Combinar datasets por indice
    df = df_features.copy()
    
    # Agregar columna de texto si existe
    if 'statement' in df_text.columns:
        df['statement'] = df_text['statement']
        text_col = 'statement'
        print(f"Columna de texto agregada: {text_col}")
    else:
        text_col = None
        print("No se encontro columna de texto")
    
    # Identificar columna target
    if 'BinaryNumTarget' in df.columns:
        target_col = 'BinaryNumTarget'
    elif 'label' in df.columns:
        target_col = 'label'
    else:
        # Buscar primera columna que contenga 'target'
        target_candidates = [col for col in df.columns if 'target' in col.lower()]
        if target_candidates:
            target_col = target_candidates[0]
        else:
            raise ValueError("No se encontro columna target")
    
    print(f"Usando target: {target_col}")
    print(f"Usando texto: {text_col}")
    
    # Eliminar nulos
    initial_size = len(df)
    if text_col:
        df = df.dropna(subset=[target_col, text_col]).copy()
    else:
        df = df.dropna(subset=[target_col]).copy()
    
    print(f"Filas despues de eliminar nulos: {len(df)} (eliminadas: {initial_size - len(df)})")
    
    # Aplicar correccion de data leakage - eliminar duplicados
    if text_col and text_col in df.columns:
        initial_size = len(df)
        df_clean = df.drop_duplicates(subset=[text_col], keep='first')
        final_size = len(df_clean)
        duplicates_removed = initial_size - final_size
        print(f"Data leakage corregido: eliminados {duplicates_removed:,} duplicados de statements")
        df = df_clean
    
    # Verificar distribucion target
    print(f"\nDistribucion {target_col}:")
    print(df[target_col].value_counts())
    print(f"Proporciones: {df[target_col].value_counts(normalize=True).round(3)}")
    
    return df, target_col, text_col

# Preparar datos
df_clean, target_col, text_col = prepare_ensemble_data()

# Visualizar distribucion
if target_col in df_clean.columns:
    fig = px.pie(values=df_clean[target_col].value_counts().values,
                 names=df_clean[target_col].value_counts().index,
                 title=f"Distribucion {target_col}")
    fig.show()

print(f"\nDataset preparado: {df_clean.shape}")
print(f"Columnas disponibles: {len(df_clean.columns)}")

Columna de texto agregada: statement
Usando target: BinaryNumTarget
Usando texto: statement
Filas despues de eliminar nulos: 134198 (eliminadas: 0)
Data leakage corregido: eliminados 133,140 duplicados de statements

Distribucion BinaryNumTarget:
BinaryNumTarget
1.0    579
0.0    479
Name: count, dtype: int64
Proporciones: BinaryNumTarget
1.0    0.547
0.0    0.453
Name: proportion, dtype: float64



Dataset preparado: (1058, 59)
Columnas disponibles: 59


## Extraccion de Features para Ensemble

Extraigo multiples tipos de caracteristicas para alimentar diferentes modelos del ensemble.

In [5]:
# Extractor de features completo
class EnsembleFeatureExtractor:
    """Extrae multiples tipos de features para ensemble"""
    
    def __init__(self):
        self.tfidf_vectorizer = None
        self.scaler = None
        self.text_features = None
        self.numeric_features = None
        
    def preprocess_text(self, text):
        """Preprocesa texto"""
        if pd.isna(text):
            return ""
        
        text = str(text).lower()
        text = re.sub(r'[^a-zA-Z0-9\s\.,;:!?]', '', text)
        text = re.sub(r'\s+', ' ', text)
        text = text.strip()
        
        return text
    
    def extract_text_features(self, texts, fit=True):
        """Extrae features TF-IDF del texto"""
        
        if texts is None or len(texts) == 0:
            return None
        
        # Preprocesar textos
        processed_texts = [self.preprocess_text(text) for text in texts]
        
        if fit:
            # Crear y ajustar vectorizador
            self.tfidf_vectorizer = TfidfVectorizer(
                max_features=5000,
                stop_words='english',
                ngram_range=(1, 2),
                min_df=2,
                max_df=0.95
            )
            text_features = self.tfidf_vectorizer.fit_transform(processed_texts)
        else:
            if self.tfidf_vectorizer is None:
                raise ValueError("Vectorizador no ajustado")
            text_features = self.tfidf_vectorizer.transform(processed_texts)
        
        return text_features
    
    def extract_numeric_features(self, df, target_col, fit=True):
        """Extrae features numericas disponibles"""
        
        # Identificar columnas numericas (excluyendo target)
        numeric_columns = df.select_dtypes(include=[np.number]).columns.tolist()
        if target_col in numeric_columns:
            numeric_columns.remove(target_col)
        
        if len(numeric_columns) == 0:
            print("No se encontraron features numericas - creando features sinteticas")
            # Crear features sinteticas basadas en texto si disponible
            if text_col and text_col in df.columns:
                df['text_length'] = df[text_col].astype(str).str.len()
                df['word_count'] = df[text_col].astype(str).str.split().str.len()
                df['char_per_word'] = df['text_length'] / (df['word_count'] + 1)
                df['exclamation_count'] = df[text_col].astype(str).str.count('!')
                df['question_count'] = df[text_col].astype(str).str.count('\?')
                
                numeric_columns = ['text_length', 'word_count', 'char_per_word', 
                                 'exclamation_count', 'question_count']
            else:
                # Features completamente sinteticas
                np.random.seed(42)
                df['synthetic_1'] = np.random.randn(len(df))
                df['synthetic_2'] = np.random.randn(len(df))
                df['synthetic_3'] = np.random.randn(len(df))
                
                numeric_columns = ['synthetic_1', 'synthetic_2', 'synthetic_3']
        
        print(f"Features numericas: {numeric_columns}")
        
        # Extraer y escalar features numericas
        numeric_data = df[numeric_columns].fillna(0).values
        
        if fit:
            self.scaler = StandardScaler()
            numeric_features = self.scaler.fit_transform(numeric_data)
        else:
            if self.scaler is None:
                raise ValueError("Scaler no ajustado")
            numeric_features = self.scaler.transform(numeric_data)
        
        return numeric_features, numeric_columns
    
    def extract_all_features(self, df, target_col, text_col=None, fit=True):
        """Extrae todos los tipos de features"""
        
        features = {}
        
        # Features numericas
        numeric_features, numeric_cols = self.extract_numeric_features(df, target_col, fit=fit)
        features['numeric'] = numeric_features
        features['numeric_columns'] = numeric_cols
        
        # Features de texto si disponible
        if text_col and text_col in df.columns:
            text_features = self.extract_text_features(df[text_col].values, fit=fit)
            if text_features is not None:
                features['text'] = text_features
        
        return features

# Crear extractor y procesar features
feature_extractor = EnsembleFeatureExtractor()

print("Extrayendo features para ensemble...")
all_features = feature_extractor.extract_all_features(df_clean, target_col, text_col, fit=True)

print(f"\nFeatures extraidas:")
for feat_type, feat_data in all_features.items():
    if feat_type != 'numeric_columns':
        if hasattr(feat_data, 'shape'):
            print(f"  {feat_type}: {feat_data.shape}")
        else:
            print(f"  {feat_type}: {type(feat_data)}")

print(f"Features numericas utilizadas: {all_features['numeric_columns']}")

Extrayendo features para ensemble...
Features numericas: ['followers_count', 'friends_count', 'favourites_count', 'statuses_count', 'listed_count', 'BotScore', 'BotScoreBinary', 'cred', 'normalize_influence', 'mentions', 'quotes', 'replies', 'retweets', 'favourites', 'hashtags', 'URLs', 'unique_count', 'total_count', 'ORG_percentage', 'NORP_percentage', 'GPE_percentage', 'PERSON_percentage', 'MONEY_percentage', 'DATE_percentage', 'CARDINAL_percentage', 'PERCENT_percentage', 'ORDINAL_percentage', 'FAC_percentage', 'LAW_percentage', 'PRODUCT_percentage', 'EVENT_percentage', 'TIME_percentage', 'LOC_percentage', 'WORK_OF_ART_percentage', 'QUANTITY_percentage', 'LANGUAGE_percentage', 'Word count', 'Max word length', 'Min word length', 'Average word length', 'present_verbs', 'past_verbs', 'adjectives', 'adverbs', 'adpositions', 'pronouns', 'TOs', 'determiners', 'conjunctions', 'dots', 'exclamation', 'questions', 'ampersand', 'capitals', 'digits', 'long_word_freq', 'short_word_freq']

Feature

## Division de Datos y Preparacion

Divido los datos en conjuntos de entrenamiento y prueba para entrenar el ensemble.

In [6]:
# Preparar datos para division
y = df_clean[target_col].values.astype(int)

# Crear matrices de features combinadas
if 'text' in all_features:
    # Combinar features numericas y texto
    from scipy.sparse import hstack, csr_matrix
    
    numeric_sparse = csr_matrix(all_features['numeric'])
    X_combined = hstack([all_features['text'], numeric_sparse])
    print(f"Features combinadas (texto + numericas): {X_combined.shape}")
else:
    # Solo features numericas
    X_combined = all_features['numeric']
    print(f"Features numericas solamente: {X_combined.shape}")

# Division train/test estratificada
X_train, X_test, y_train, y_test = train_test_split(
    X_combined, y, test_size=0.2, random_state=42, stratify=y
)

print(f"\nDivision de datos:")
print(f"Entrenamiento: {X_train.shape[0]} muestras, {X_train.shape[1]} features")
print(f"Prueba: {X_test.shape[0]} muestras, {X_test.shape[1]} features")

print(f"\nDistribucion entrenamiento: {np.bincount(y_train)}")
print(f"Distribucion prueba: {np.bincount(y_test)}")

# Verificar balance
train_balance = np.bincount(y_train) / len(y_train)
test_balance = np.bincount(y_test) / len(y_test)

print(f"\nBalance entrenamiento: {train_balance.round(3)}")
print(f"Balance prueba: {test_balance.round(3)}")

# Preparar features solo numericas para algunos algoritmos
X_train_numeric = all_features['numeric'][:X_train.shape[0]]
X_test_numeric = all_features['numeric'][X_train.shape[0]:]

print(f"\nFeatures numericas separadas preparadas")
print(f"Train numericas: {X_train_numeric.shape}")
print(f"Test numericas: {X_test_numeric.shape}")

Features combinadas (texto + numericas): (1058, 1848)

Division de datos:
Entrenamiento: 846 muestras, 1848 features
Prueba: 212 muestras, 1848 features

Distribucion entrenamiento: [383 463]
Distribucion prueba: [ 96 116]

Balance entrenamiento: [0.453 0.547]
Balance prueba: [0.453 0.547]

Features numericas separadas preparadas
Train numericas: (846, 57)
Test numericas: (212, 57)


## Algoritmos Base para Ensemble

Defino y configuro todos los algoritmos base que integrare en el sistema de ensemble.

In [7]:
# Definir algoritmos base optimizados basados en mejores resultados de notebooks 03-06
def create_base_algorithms():
    """Crea conjunto de mejores algoritmos de cada notebook"""
    
    # Mejores modelos lineales (Notebook 04)
    linear_models = {
        'ridge_cv': RidgeClassifier(random_state=42, alpha=1.0),  # Mejor: 0.6554
        'logistic_elastic': LogisticRegression(random_state=42, penalty='elasticnet', 
                                             C=1.0, l1_ratio=0.5, solver='saga', max_iter=1000),  # 0.6552
        'ridge_basic': RidgeClassifier(random_state=42, alpha=1.0)  # 0.6547
    }
    
    # Mejores boosting (Notebook 05) - Los que mas F1-Score dieron
    boosting_models = {
        'xgboost_aggressive': xgb.XGBClassifier(  # Mejor: 0.7150
            n_estimators=200, learning_rate=0.15, max_depth=6, 
            subsample=0.9, colsample_bytree=0.9, gamma=0.1,
            random_state=42, eval_metric='logloss'
        ),
        'lightgbm_balanced': lgb.LGBMClassifier(  # 0.6991
            n_estimators=150, learning_rate=0.08, max_depth=6,
            colsample_bytree=0.9, subsample=0.9, reg_alpha=0.1, reg_lambda=0.1,
            random_state=42, verbose=-1
        ),
        'catboost_optimized': None  # Se agregará después de importar
    }
    
    # Mejores tradicionales (Notebook 03)
    traditional_models = {
        'random_forest_best': RandomForestClassifier(  # Mejor: 0.6749
            n_estimators=100, random_state=42, max_depth=15, min_samples_split=5
        ),
        'extra_trees_best': ExtraTreesClassifier(  # 0.6428
            n_estimators=100, random_state=42, max_depth=15, min_samples_split=5
        )
    }
    
    # Mejores otros algoritmos (Notebook 06)
    other_models = {
        'gaussian_nb_smooth': GaussianNB(var_smoothing=1e-08),  # Mejor: 0.6822
        'knn_15': KNeighborsClassifier(n_neighbors=15, weights='distance'),  # 0.6665
        'lda_best': LinearDiscriminantAnalysis(),  # 0.6547
        'svm_linear_l1': SVC(kernel='linear', C=1.0, probability=True, random_state=42)  # 0.6552
    }
    
    # Combinar todos los modelos
    all_models = {}
    all_models.update(linear_models)
    all_models.update(boosting_models)
    all_models.update(traditional_models)
    all_models.update(other_models)
    
    # Separar por compatibilidad con sparse matrices
    sparse_compatible = {
        'logistic_elastic': all_models['logistic_elastic'],
        'ridge_cv': all_models['ridge_cv'],
        'ridge_basic': all_models['ridge_basic'],
        'svm_linear_l1': all_models['svm_linear_l1']
    }
    
    dense_only = {
        'xgboost_aggressive': all_models['xgboost_aggressive'],
        'lightgbm_balanced': all_models['lightgbm_balanced'],
        'random_forest_best': all_models['random_forest_best'],
        'extra_trees_best': all_models['extra_trees_best'],
        'gaussian_nb_smooth': all_models['gaussian_nb_smooth'],
        'knn_15': all_models['knn_15'],
        'lda_best': all_models['lda_best']
    }
    
    # Agregar CatBoost si está disponible
    try:
        import catboost as cb
        dense_only['catboost_optimized'] = cb.CatBoostClassifier(
            iterations=150, learning_rate=0.08, depth=6, l2_leaf_reg=1,
            random_seed=42, verbose=False, bootstrap_type='Bayesian'
        )
    except ImportError:
        print("CatBoost no disponible, se omite del ensemble")
    
    return sparse_compatible, dense_only

# Crear algoritmos
sparse_algorithms, dense_algorithms = create_base_algorithms()

print(f"Algoritmos compatibles con sparse matrices: {len(sparse_algorithms)}")
for name in sparse_algorithms.keys():
    print(f"  - {name}")

print(f"\nAlgoritmos que requieren arrays densos: {len(dense_algorithms)}")
for name in dense_algorithms.keys():
    print(f"  - {name}")

print(f"\nTotal algoritmos base: {len(sparse_algorithms) + len(dense_algorithms)}")

Algoritmos compatibles con sparse matrices: 4
  - logistic_elastic
  - ridge_cv
  - ridge_basic
  - svm_linear_l1

Algoritmos que requieren arrays densos: 8
  - xgboost_aggressive
  - lightgbm_balanced
  - random_forest_best
  - extra_trees_best
  - gaussian_nb_smooth
  - knn_15
  - lda_best
  - catboost_optimized

Total algoritmos base: 12


## Evaluacion Individual de Algoritmos

Evaluo cada algoritmo individualmente para determinar su rendimiento y seleccionar los mejores para el ensemble.

In [8]:
# Evaluador de algoritmos individuales
def evaluate_algorithm(name, algorithm, X_train, X_test, y_train, y_test, use_sparse=True):
    """Evalua un algoritmo individual con proteccion de namespace"""
    
    start_time = time.time()
    
    try:
        # Seleccionar tipo de datos
        if use_sparse:
            X_tr, X_te = X_train, X_test
        else:
            # Convertir a denso si es necesario
            if hasattr(X_train, 'toarray'):
                X_tr, X_te = X_train.toarray(), X_test.toarray()
            else:
                X_tr, X_te = X_train, X_test
        
        # Entrenar modelo
        algorithm.fit(X_tr, y_train)
        
        # Predicciones
        y_pred = algorithm.predict(X_te)
        
        # Probabilidades
        if hasattr(algorithm, 'predict_proba'):
            y_pred_proba = algorithm.predict_proba(X_te)[:, 1]
        elif hasattr(algorithm, 'decision_function'):
            from sklearn.preprocessing import MinMaxScaler
            decision_scores = algorithm.decision_function(X_te)
            scaler = MinMaxScaler()
            y_pred_proba = scaler.fit_transform(decision_scores.reshape(-1, 1)).flatten()
        else:
            y_pred_proba = y_pred.astype(float)
        
        # Calcular metricas usando importacion directa para evitar namespace conflicts
        import sklearn.metrics
        accuracy_eval = sklearn.metrics.accuracy_score(y_test, y_pred)
        f1_eval = sklearn.metrics.f1_score(y_test, y_pred, average='weighted')
        
        try:
            roc_auc_eval = sklearn.metrics.roc_auc_score(y_test, y_pred_proba)
        except:
            roc_auc_eval = 0.0
        
        # Validacion cruzada rapida
        try:
            cv_scores = cross_val_score(
                algorithm, X_tr, y_train, cv=3, scoring='f1_weighted', n_jobs=-1
            )
            cv_mean = cv_scores.mean()
            cv_std = cv_scores.std()
        except:
            cv_mean = cv_std = 0.0
        
        training_time = time.time() - start_time
        
        result = {
            'name': name,
            'accuracy': accuracy_eval,
            'f1_score': f1_eval,
            'roc_auc': roc_auc_eval,
            'cv_f1_mean': cv_mean,
            'cv_f1_std': cv_std,
            'training_time': training_time,
            'success': True,
            'model': algorithm
        }
        
        print(f"  {name}: F1={f1_eval:.3f}, ROC-AUC={roc_auc_eval:.3f}, Time={training_time:.2f}s")
        return result
        
    except Exception as e:
        print(f"  {name}: ERROR - {str(e)[:50]}")
        return {
            'name': name,
            'accuracy': 0.0,
            'f1_score': 0.0,
            'roc_auc': 0.0,
            'cv_f1_mean': 0.0,
            'cv_f1_std': 0.0,
            'training_time': 0.0,
            'success': False,
            'model': None
        }

# Evaluar todos los algoritmos
print("Evaluando algoritmos individuales...")
print("=" * 60)

individual_results = []

# Evaluar algoritmos compatibles con sparse
print("\nAlgoritmos sparse-compatible:")
for name, algorithm in sparse_algorithms.items():
    result = evaluate_algorithm(name, algorithm, X_train, X_test, y_train, y_test, use_sparse=True)
    individual_results.append(result)

# Evaluar algoritmos que requieren denso
print("\nAlgoritmos dense-only:")
for name, algorithm in dense_algorithms.items():
    result = evaluate_algorithm(name, algorithm, X_train, X_test, y_train, y_test, use_sparse=False)
    individual_results.append(result)

print("\n" + "=" * 60)
print("Evaluacion individual completada")

Evaluando algoritmos individuales...

Algoritmos sparse-compatible:
  logistic_elastic: F1=0.722, ROC-AUC=0.810, Time=5.16s
  ridge_cv: F1=0.811, ROC-AUC=0.882, Time=2.30s
  ridge_basic: F1=0.811, ROC-AUC=0.882, Time=2.39s
  svm_linear_l1: F1=0.807, ROC-AUC=0.872, Time=3.68s

Algoritmos dense-only:
  xgboost_aggressive: F1=0.777, ROC-AUC=0.858, Time=2.12s
  lightgbm_balanced: F1=0.782, ROC-AUC=0.852, Time=4.91s
  random_forest_best: F1=0.764, ROC-AUC=0.840, Time=0.90s
  extra_trees_best: F1=0.792, ROC-AUC=0.887, Time=0.99s
  gaussian_nb_smooth: F1=0.741, ROC-AUC=0.755, Time=0.25s
  knn_15: F1=0.626, ROC-AUC=0.673, Time=0.46s
  lda_best: F1=0.689, ROC-AUC=0.749, Time=1.34s
  catboost_optimized: F1=0.795, ROC-AUC=0.841, Time=5.85s

Evaluacion individual completada


In [9]:
# Analizar resultados individuales
successful_results = [r for r in individual_results if r['success']]

if len(successful_results) > 0:
    # Crear DataFrame de resultados
    results_df = pd.DataFrame(successful_results)
    results_df = results_df.sort_values('f1_score', ascending=False)
    
    print(f"\nResultados de algoritmos individuales ({len(successful_results)} exitosos):")
    print(results_df[['name', 'accuracy', 'f1_score', 'roc_auc', 'cv_f1_mean', 'training_time']].round(4))
    
    # Mejores algoritmos
    top_5 = results_df.head(5)
    print(f"\nTop 5 algoritmos:")
    for i, (idx, row) in enumerate(top_5.iterrows(), 1):
        print(f"  {i}. {row['name']}: F1={row['f1_score']:.4f}, CV={row['cv_f1_mean']:.4f}")
    
    # Estadisticas generales
    print(f"\nEstadisticas generales:")
    print(f"  F1-Score promedio: {results_df['f1_score'].mean():.4f}")
    print(f"  Mejor F1-Score: {results_df['f1_score'].max():.4f}")
    print(f"  Tiempo promedio: {results_df['training_time'].mean():.2f}s")
    
    # Seleccionar modelos para ensemble (F1 > 0.6)
    good_models = results_df[results_df['f1_score'] >= 0.6]
    if len(good_models) == 0:
        good_models = results_df.head(max(3, len(results_df)//2))  # Al menos 3 o la mitad
    
    print(f"\nModelos seleccionados para ensemble: {len(good_models)}")
    for _, row in good_models.iterrows():
        print(f"  - {row['name']}: F1={row['f1_score']:.4f}")
else:
    print("No se evaluaron algoritmos exitosamente")
    results_df = None
    good_models = None


Resultados de algoritmos individuales (12 exitosos):
                  name  accuracy  f1_score  roc_auc  cv_f1_mean  training_time
1             ridge_cv    0.8113    0.8115   0.8816      0.7682         2.2982
2          ridge_basic    0.8113    0.8115   0.8816      0.7682         2.3907
3        svm_linear_l1    0.8066    0.8068   0.8720      0.7568         3.6844
11  catboost_optimized    0.7972    0.7947   0.8405      0.7452         5.8463
7     extra_trees_best    0.7972    0.7919   0.8869      0.7308         0.9881
5    lightgbm_balanced    0.7830    0.7823   0.8523      0.6998         4.9109
4   xgboost_aggressive    0.7783    0.7774   0.8583      0.7042         2.1160
6   random_forest_best    0.7689    0.7643   0.8398      0.7279         0.9002
8   gaussian_nb_smooth    0.7406    0.7411   0.7552      0.7352         0.2537
0     logistic_elastic    0.7217    0.7216   0.8096      0.6982         5.1559
10            lda_best    0.6887    0.6893   0.7495      0.6824         1.343

## Visualizacion de Rendimiento Individual

Visualizo el rendimiento de cada algoritmo para analizar fortalezas y debilidades.

In [10]:
# Visualizaciones de rendimiento individual
if results_df is not None and len(results_df) > 0:
    # Dashboard de rendimiento
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=['F1-Score por Algoritmo', 'ROC-AUC vs Accuracy', 
                       'Tiempo de Entrenamiento', 'Validacion Cruzada']
    )
    
    # 1. F1-Score por algoritmo
    fig.add_trace(
        go.Bar(
            x=results_df['name'],
            y=results_df['f1_score'],
            name='F1-Score',
            text=[f'{f:.3f}' for f in results_df['f1_score']],
            textposition='outside',
            marker_color='lightblue'
        ),
        row=1, col=1
    )
    
    # 2. ROC-AUC vs Accuracy
    fig.add_trace(
        go.Scatter(
            x=results_df['accuracy'],
            y=results_df['roc_auc'],
            mode='markers+text',
            text=results_df['name'],
            textposition='top center',
            name='AUC vs Accuracy',
            marker=dict(
                size=results_df['f1_score'] * 15,
                color=results_df['f1_score'],
                colorscale='Viridis',
                showscale=True,
                colorbar=dict(title="F1-Score")
            )
        ),
        row=1, col=2
    )
    
    # 3. Tiempo de entrenamiento
    fig.add_trace(
        go.Bar(
            x=results_df['name'],
            y=results_df['training_time'],
            name='Tiempo (s)',
            text=[f'{t:.2f}s' for t in results_df['training_time']],
            textposition='outside',
            marker_color='lightcoral'
        ),
        row=2, col=1
    )
    
    # 4. Validacion cruzada
    fig.add_trace(
        go.Bar(
            x=results_df['name'],
            y=results_df['cv_f1_mean'],
            error_y=dict(type='data', array=results_df['cv_f1_std']),
            name='CV F1-Score',
            text=[f'{cv:.3f}±{std:.3f}' for cv, std in 
                 zip(results_df['cv_f1_mean'], results_df['cv_f1_std'])],
            textposition='outside',
            marker_color='lightgreen'
        ),
        row=2, col=2
    )
    
    # Configuracion del layout
    fig.update_layout(
        title='Rendimiento Individual de Algoritmos - Ensemble Completo',
        height=800,
        showlegend=False
    )
    
    # Actualizar ejes
    fig.update_xaxes(tickangle=45, row=1, col=1)
    fig.update_xaxes(tickangle=45, row=2, col=1)
    fig.update_xaxes(tickangle=45, row=2, col=2)
    
    fig.show()
    
    # Matriz de correlacion de metricas
    metrics_cols = ['accuracy', 'f1_score', 'roc_auc', 'cv_f1_mean', 'training_time']
    corr_matrix = results_df[metrics_cols].corr()
    
    fig_corr = px.imshow(
        corr_matrix,
        text_auto=".3f",
        aspect="auto",
        title="Correlacion entre Metricas de Rendimiento",
        color_continuous_scale="RdBu_r"
    )
    fig_corr.show()
    
else:
    print("No hay datos para visualizar")

## Construccion del Ensemble

Construyo diferentes tipos de ensemble usando los mejores algoritmos identificados.

In [11]:
# Constructor de ensembles
class EnsembleBuilder:
    """Construye diferentes tipos de ensemble"""

    def __init__(self, good_models_df):
        self.good_models = good_models_df
        self.ensembles = {}
        self.ensemble_results = {}

    def calculate_weights(self):
        """Calcula pesos basados en rendimiento"""

        try:
            # Peso base por F1-score
            f1_scores = self.good_models['f1_score'].values
            f1_weights = f1_scores / f1_scores.sum()

            # Bonus por estabilidad (menor std en CV)
            stability_scores = 1 / (1 + self.good_models['cv_f1_std'].values)
            stability_weights = stability_scores / stability_scores.sum()

            # Combinar pesos
            final_weights = 0.7 * f1_weights + 0.3 * stability_weights

            # Normalizar
            final_weights = final_weights / final_weights.sum()

            return final_weights

        except Exception as e:
            print(f"Error calculando pesos: {e}")
            # Retornar pesos uniformes como fallback
            n_models = len(self.good_models)
            return np.ones(n_models) / n_models

    def create_voting_ensemble(self, X_train, X_test, y_train, y_test):
        """Crea ensemble de votacion solo con modelos que tienen predict_proba"""

        if len(self.good_models) < 2:
            print("Se necesitan al menos 2 modelos para voting")
            return None

        try:
            # Preparar estimadores que tengan predict_proba
            estimators = []

            for idx, (_, row) in enumerate(self.good_models.iterrows()):
                if row['model'] is not None and hasattr(row['model'], 'predict_proba'):
                    estimators.append((row['name'], row['model']))

            if len(estimators) < 2:
                print("No hay suficientes modelos con predict_proba para voting")
                return None

            # Calcular pesos solo para modelos seleccionados
            weights = self.calculate_weights()[:len(estimators)]

            # Crear ensemble de votacion suave
            voting_ensemble = VotingClassifier(
                estimators=estimators,
                voting='soft',
                weights=weights
            )

            print(f"Voting ensemble creado con {len(estimators)} modelos")
            return voting_ensemble

        except Exception as e:
            print(f"Error creando voting ensemble: {e}")
            return None

    def create_stacking_ensemble(self, X_train, X_test, y_train, y_test):
        """Crea ensemble de stacking"""

        if len(self.good_models) < 2:
            print("Se necesitan al menos 2 modelos para stacking")
            return None

        try:
            # Preparar estimadores base
            estimators = []
            for _, row in self.good_models.iterrows():
                if row['model'] is not None:
                    estimators.append((row['name'], row['model']))

            if len(estimators) < 2:
                print("No hay suficientes modelos validos para stacking")
                return None

            # Meta-learner
            meta_learner = LogisticRegression(random_state=42, max_iter=1000)

            # Crear stacking ensemble
            stacking_ensemble = StackingClassifier(
                estimators=estimators,
                final_estimator=meta_learner,
                cv=3,
                passthrough=False,
                n_jobs=-1
            )

            print(f"Stacking ensemble creado con {len(estimators)} modelos")
            return stacking_ensemble

        except Exception as e:
            print(f"Error creando stacking ensemble: {e}")
            return None

    def create_bagging_ensemble(self, X_train, X_test, y_train, y_test):
        """Crea ensemble de bagging con mejor modelo base"""

        # Usar el mejor modelo como base
        best_model_row = self.good_models.iloc[0]
        best_model = best_model_row['model']

        if best_model is None:
            print("No hay modelo base valido para bagging")
            return None

        try:
            # Crear bagging ensemble usando estimator en lugar de base_estimator
            bagging_ensemble = BaggingClassifier(
                estimator=best_model,
                n_estimators=50,
                random_state=42,
                n_jobs=-1
            )

            print(f"Bagging ensemble creado con {best_model_row['name']} como base")
            return bagging_ensemble
        except Exception as e:
            print(f"Error creando bagging ensemble: {e}")
            return None

    def build_all_ensembles(self, X_train, X_test, y_train, y_test):
        """Construye todos los tipos de ensemble"""

        print("Construyendo ensembles...")

        # Voting ensemble
        voting_ens = self.create_voting_ensemble(X_train, X_test, y_train, y_test)
        if voting_ens is not None:
            self.ensembles['voting'] = voting_ens

        # Stacking ensemble
        stacking_ens = self.create_stacking_ensemble(X_train, X_test, y_train, y_test)
        if stacking_ens is not None:
            self.ensembles['stacking'] = stacking_ens

        # Bagging ensemble
        bagging_ens = self.create_bagging_ensemble(X_train, X_test, y_train, y_test)
        if bagging_ens is not None:
            self.ensembles['bagging'] = bagging_ens

        print(f"\n{len(self.ensembles)} ensembles construidos:")
        for name in self.ensembles.keys():
            print(f"  - {name}")

        return len(self.ensembles)

# Construir ensembles si tenemos buenos modelos
if good_models is not None and len(good_models) > 0:
    ensemble_builder = EnsembleBuilder(good_models)
    num_ensembles = ensemble_builder.build_all_ensembles(X_train, X_test, y_train, y_test)

    if num_ensembles > 0:
        print("\nEnsembles construidos exitosamente")
    else:
        print("\nNo se pudieron construir ensembles")
else:
    print("No hay modelos suficientemente buenos para construir ensembles")
    ensemble_builder = None

Construyendo ensembles...
Voting ensemble creado con 10 modelos
Stacking ensemble creado con 12 modelos
Bagging ensemble creado con ridge_cv como base

3 ensembles construidos:
  - voting
  - stacking
  - bagging

Ensembles construidos exitosamente


## Entrenamiento y Evaluacion de Ensembles

Entreno y evaluo todos los ensembles construidos para comparar su rendimiento.

In [12]:
# Evaluador de ensembles con proteccion de namespace
def evaluate_ensemble(name, ensemble, X_train, X_test, y_train, y_test):
    """Evalua un ensemble completo con proteccion de namespace"""
    
    start_time = time.time()
    
    try:
        print(f"\nEntrenando {name} ensemble...")
        
        # Determinar tipo de datos segun ensemble
        if hasattr(X_train, 'toarray'):  # Es sparse
            # Algunos ensembles pueden manejar sparse, otros no
            try:
                ensemble.fit(X_train, y_train)
                y_pred = ensemble.predict(X_test)
                use_sparse = True
            except:
                # Convertir a denso si falla
                X_tr_dense = X_train.toarray()
                X_te_dense = X_test.toarray()
                ensemble.fit(X_tr_dense, y_train)
                y_pred = ensemble.predict(X_te_dense)
                use_sparse = False
                print(f"  {name}: Convertido a arrays densos")
        else:
            ensemble.fit(X_train, y_train)
            y_pred = ensemble.predict(X_test)
            use_sparse = False
        
        # Probabilidades
        if hasattr(ensemble, 'predict_proba'):
            if use_sparse:
                y_pred_proba = ensemble.predict_proba(X_test)[:, 1]
            else:
                y_pred_proba = ensemble.predict_proba(X_te_dense if 'X_te_dense' in locals() else X_test)[:, 1]
        else:
            y_pred_proba = y_pred.astype(float)
        
        # Calcular metricas usando importacion directa para evitar namespace conflicts
        import sklearn.metrics
        accuracy_eval = sklearn.metrics.accuracy_score(y_test, y_pred)
        f1_eval = sklearn.metrics.f1_score(y_test, y_pred, average='weighted')
        
        try:
            roc_auc_eval = sklearn.metrics.roc_auc_score(y_test, y_pred_proba)
        except:
            roc_auc_eval = 0.0
        
        # Validacion cruzada
        try:
            if use_sparse:
                cv_scores = cross_val_score(
                    ensemble, X_train, y_train, cv=3, scoring='f1_weighted', n_jobs=-1
                )
            else:
                cv_scores = cross_val_score(
                    ensemble, X_tr_dense if 'X_tr_dense' in locals() else X_train, 
                    y_train, cv=3, scoring='f1_weighted', n_jobs=-1
                )
            cv_mean = cv_scores.mean()
            cv_std = cv_scores.std()
        except:
            cv_mean = cv_std = 0.0
        
        training_time = time.time() - start_time
        
        # Metricas de confianza
        confidence = np.abs(y_pred_proba - 0.5) * 2  # Normalizado 0-1
        avg_confidence = confidence.mean()
        
        high_conf_mask = confidence >= 0.7
        high_conf_accuracy = sklearn.metrics.accuracy_score(
            y_test[high_conf_mask], y_pred[high_conf_mask]
        ) if np.sum(high_conf_mask) > 0 else 0.0
        
        result = {
            'name': name,
            'accuracy': accuracy_eval,
            'f1_score': f1_eval,
            'roc_auc': roc_auc_eval,
            'cv_f1_mean': cv_mean,
            'cv_f1_std': cv_std,
            'training_time': training_time,
            'avg_confidence': avg_confidence,
            'high_conf_samples': np.sum(high_conf_mask),
            'high_conf_accuracy': high_conf_accuracy,
            'predictions': y_pred,
            'probabilities': y_pred_proba,
            'confidence_scores': confidence,
            'success': True
        }
        
        print(f"  {name}: F1={f1_eval:.4f}, AUC={roc_auc_eval:.4f}, Conf={avg_confidence:.3f}")
        return result
        
    except Exception as e:
        print(f"  {name}: ERROR - {str(e)}")
        return {
            'name': name,
            'success': False,
            'error': str(e)
        }

# Evaluar todos los ensembles
if ensemble_builder and len(ensemble_builder.ensembles) > 0:
    print("Evaluando ensembles...")
    print("=" * 50)
    
    ensemble_results = []
    
    for name, ensemble in ensemble_builder.ensembles.items():
        result = evaluate_ensemble(name, ensemble, X_train, X_test, y_train, y_test)
        ensemble_results.append(result)
    
    print("\n" + "=" * 50)
    print("Evaluacion de ensembles completada")
    
    # Filtrar resultados exitosos
    successful_ensembles = [r for r in ensemble_results if r['success']]
    
    if len(successful_ensembles) > 0:
        ensemble_df = pd.DataFrame(successful_ensembles)
        ensemble_df = ensemble_df.sort_values('f1_score', ascending=False)
        
        print(f"\nResultados de ensembles:")
        display_cols = ['name', 'accuracy', 'f1_score', 'roc_auc', 'avg_confidence', 'training_time']
        print(ensemble_df[display_cols].round(4))
        
        # Mejor ensemble
        best_ensemble = ensemble_df.iloc[0]
        print(f"\nMejor ensemble: {best_ensemble['name']}")
        print(f"  F1-Score: {best_ensemble['f1_score']:.4f}")
        print(f"  Accuracy: {best_ensemble['accuracy']:.4f}")
        print(f"  ROC-AUC: {best_ensemble['roc_auc']:.4f}")
        print(f"  Confianza promedio: {best_ensemble['avg_confidence']:.3f}")
    else:
        print("No se evaluaron ensembles exitosamente")
        ensemble_df = None
else:
    print("No hay ensembles para evaluar")
    ensemble_results = []
    ensemble_df = None

Evaluando ensembles...

Entrenando voting ensemble...
  voting: Convertido a arrays densos
  voting: F1=0.8296, AUC=0.8974, Conf=0.417

Entrenando stacking ensemble...
  stacking: Convertido a arrays densos
  stacking: F1=0.8304, AUC=0.9011, Conf=0.630

Entrenando bagging ensemble...
  bagging: F1=0.8017, AUC=0.8830, Conf=0.804

Evaluacion de ensembles completada

Resultados de ensembles:
       name  accuracy  f1_score  roc_auc  avg_confidence  training_time
1  stacking    0.8302    0.8304   0.9011          0.6296        92.3475
0    voting    0.8302    0.8296   0.8974          0.4168        47.8215
2   bagging    0.8019    0.8017   0.8830          0.8043         2.2606

Mejor ensemble: stacking
  F1-Score: 0.8304
  Accuracy: 0.8302
  ROC-AUC: 0.9011
  Confianza promedio: 0.630


## Comparacion Final: Individual vs Ensemble

Comparo el rendimiento de los ensembles contra los mejores modelos individuales.

In [13]:
# Comparacion comprehensiva
def compare_all_approaches():
    """Compara todos los enfoques: individuales vs ensembles"""
    
    if results_df is None or ensemble_df is None:
        print("Datos insuficientes para comparacion")
        return None
    
    # Mejor modelo individual
    best_individual = results_df.iloc[0]
    
    # Mejor ensemble
    best_ensemble = ensemble_df.iloc[0]
    
    print("COMPARACION FINAL: INDIVIDUAL vs ENSEMBLE")
    print("=" * 60)
    
    print(f"\nMEJOR MODELO INDIVIDUAL:")
    print(f"  Modelo: {best_individual['name']}")
    print(f"  F1-Score: {best_individual['f1_score']:.4f}")
    print(f"  Accuracy: {best_individual['accuracy']:.4f}")
    print(f"  ROC-AUC: {best_individual['roc_auc']:.4f}")
    print(f"  Tiempo: {best_individual['training_time']:.2f}s")
    
    print(f"\nMEJOR ENSEMBLE:")
    print(f"  Modelo: {best_ensemble['name']}")
    print(f"  F1-Score: {best_ensemble['f1_score']:.4f}")
    print(f"  Accuracy: {best_ensemble['accuracy']:.4f}")
    print(f"  ROC-AUC: {best_ensemble['roc_auc']:.4f}")
    print(f"  Confianza: {best_ensemble['avg_confidence']:.3f}")
    print(f"  Tiempo: {best_ensemble['training_time']:.2f}s")
    
    # Calcular mejoras
    f1_improvement = best_ensemble['f1_score'] - best_individual['f1_score']
    acc_improvement = best_ensemble['accuracy'] - best_individual['accuracy']
    auc_improvement = best_ensemble['roc_auc'] - best_individual['roc_auc']
    
    f1_improvement_pct = (f1_improvement / best_individual['f1_score']) * 100
    acc_improvement_pct = (acc_improvement / best_individual['accuracy']) * 100
    auc_improvement_pct = (auc_improvement / best_individual['roc_auc']) * 100 if best_individual['roc_auc'] > 0 else 0
    
    print(f"\nMEJORAS DEL ENSEMBLE:")
    print(f"  F1-Score: {f1_improvement:+.4f} ({f1_improvement_pct:+.2f}%)")
    print(f"  Accuracy: {acc_improvement:+.4f} ({acc_improvement_pct:+.2f}%)")
    print(f"  ROC-AUC: {auc_improvement:+.4f} ({auc_improvement_pct:+.2f}%)")
    
    # Evaluacion general
    if f1_improvement > 0.01:  # Mejora significativa
        verdict = "ENSEMBLE SUPERIOR"
    elif abs(f1_improvement) <= 0.01:  # Similar rendimiento
        verdict = "RENDIMIENTO SIMILAR"
    else:
        verdict = "INDIVIDUAL SUPERIOR"
    
    print(f"\nVEREDICTO: {verdict}")
    
    return {
        'best_individual': best_individual,
        'best_ensemble': best_ensemble,
        'f1_improvement': f1_improvement,
        'f1_improvement_pct': f1_improvement_pct,
        'verdict': verdict
    }

# Realizar comparacion
if results_df is not None and ensemble_df is not None:
    comparison_results = compare_all_approaches()
else:
    print("No se puede realizar comparacion - datos faltantes")
    comparison_results = None

COMPARACION FINAL: INDIVIDUAL vs ENSEMBLE

MEJOR MODELO INDIVIDUAL:
  Modelo: ridge_cv
  F1-Score: 0.8115
  Accuracy: 0.8113
  ROC-AUC: 0.8816
  Tiempo: 2.30s

MEJOR ENSEMBLE:
  Modelo: stacking
  F1-Score: 0.8304
  Accuracy: 0.8302
  ROC-AUC: 0.9011
  Confianza: 0.630
  Tiempo: 92.35s

MEJORAS DEL ENSEMBLE:
  F1-Score: +0.0190 (+2.34%)
  Accuracy: +0.0189 (+2.33%)
  ROC-AUC: +0.0195 (+2.21%)

VEREDICTO: ENSEMBLE SUPERIOR


## Visualizaciones Finales y Dashboard

Genero visualizaciones completas que resumen todo el analisis del ensemble.

In [14]:
# Dashboard final completo
def create_final_dashboard():
    """Crea dashboard final con todas las comparaciones"""
    
    if results_df is None:
        print("No hay datos para dashboard")
        return None
    
    # Combinar datos para visualizacion
    individual_data = results_df.copy()
    individual_data['type'] = 'Individual'
    
    if ensemble_df is not None:
        ensemble_data = ensemble_df.copy()
        ensemble_data['type'] = 'Ensemble'
        
        # Combinar datasets
        combined_data = pd.concat([individual_data, ensemble_data], ignore_index=True)
    else:
        combined_data = individual_data
    
    # Dashboard con multiples subplots
    fig = make_subplots(
        rows=3, cols=2,
        subplot_titles=[
            'F1-Score: Individual vs Ensemble',
            'ROC-AUC vs Accuracy',
            'Tiempo de Entrenamiento',
            'Validacion Cruzada',
            'Top 10 Modelos',
            'Distribucion de Metricas'
        ]
    )
    
    # 1. F1-Score comparacion
    colors = {'Individual': 'lightblue', 'Ensemble': 'lightcoral'}
    for model_type in combined_data['type'].unique():
        subset = combined_data[combined_data['type'] == model_type]
        fig.add_trace(
            go.Bar(
                x=subset['name'],
                y=subset['f1_score'],
                name=f'{model_type}',
                marker_color=colors.get(model_type, 'gray'),
                text=[f'{f:.3f}' for f in subset['f1_score']],
                textposition='outside',
                legendgroup=model_type,
                showlegend=(model_type == 'Individual')  # Solo mostrar una leyenda
            ),
            row=1, col=1
        )
    
    # 2. ROC-AUC vs Accuracy scatter
    for model_type in combined_data['type'].unique():
        subset = combined_data[combined_data['type'] == model_type]
        fig.add_trace(
            go.Scatter(
                x=subset['accuracy'],
                y=subset['roc_auc'],
                mode='markers',
                name=f'{model_type} (AUC vs Acc)',
                marker=dict(
                    size=10,
                    color=colors.get(model_type, 'gray'),
                    opacity=0.7
                ),
                text=subset['name'],
                textposition='top center',
                legendgroup=model_type,
                showlegend=False
            ),
            row=1, col=2
        )
    
    # 3. Tiempo de entrenamiento
    fig.add_trace(
        go.Bar(
            x=combined_data['name'],
            y=combined_data['training_time'],
            name='Tiempo',
            marker_color='lightgreen',
            text=[f'{t:.1f}s' for t in combined_data['training_time']],
            textposition='outside',
            showlegend=False
        ),
        row=2, col=1
    )
    
    # 4. Validacion cruzada
    fig.add_trace(
        go.Bar(
            x=combined_data['name'],
            y=combined_data['cv_f1_mean'],
            error_y=dict(type='data', array=combined_data['cv_f1_std']),
            name='CV F1',
            marker_color='lightyellow',
            showlegend=False
        ),
        row=2, col=2
    )
    
    # 5. Top 10 modelos
    top_10 = combined_data.nlargest(10, 'f1_score')
    fig.add_trace(
        go.Bar(
            x=top_10['name'],
            y=top_10['f1_score'],
            name='Top 10',
            marker_color=[colors.get(t, 'gray') for t in top_10['type']],
            text=[f'{f:.3f}' for f in top_10['f1_score']],
            textposition='outside',
            showlegend=False
        ),
        row=3, col=1
    )
    
    # 6. Box plot de metricas
    metrics = ['f1_score', 'accuracy', 'roc_auc']
    for i, metric in enumerate(metrics):
        fig.add_trace(
            go.Box(
                y=combined_data[metric],
                name=metric.replace('_', ' ').title(),
                marker_color=px.colors.qualitative.Set3[i],
                showlegend=False
            ),
            row=3, col=2
        )
    
    # Configuracion del layout
    fig.update_layout(
        title={
            'text': 'Dashboard Final - Ensemble Completo con Todos los Enfoques',
            'x': 0.5,
            'font': {'size': 20}
        },
        height=1200,
        showlegend=True
    )
    
    # Actualizar ejes
    for row in range(1, 4):
        for col in range(1, 3):
            fig.update_xaxes(tickangle=45, row=row, col=col)
    
    return fig

# Crear dashboard final
final_dashboard = create_final_dashboard()
if final_dashboard:
    final_dashboard.show()
    print("Dashboard final generado")
else:
    print("No se pudo generar dashboard final")

Dashboard final generado


In [15]:
# Matriz de confusion del mejor modelo
if comparison_results:
    best_model_data = comparison_results['best_ensemble'] if comparison_results['verdict'] != 'INDIVIDUAL SUPERIOR' else comparison_results['best_individual']
    
    if 'predictions' in best_model_data:
        y_pred_best = best_model_data['predictions']
        
        # Matriz de confusion
        cm = confusion_matrix(y_test, y_pred_best)
        
        fig_cm = px.imshow(
            cm,
            labels=dict(x="Prediccion", y="Verdadero", color="Cantidad"),
            x=['Fake', 'Real'],
            y=['Fake', 'Real'],
            color_continuous_scale='Blues',
            title=f"Matriz de Confusion - {best_model_data['name']}"
        )
        
        # Añadir texto a cada celda
        for i in range(2):
            for j in range(2):
                fig_cm.add_annotation(
                    x=j, y=i,
                    text=str(cm[i, j]),
                    showarrow=False,
                    font=dict(color="white" if cm[i, j] > cm.max()/2 else "black", size=16)
                )
        
        fig_cm.show()
        
        # Calcular metricas de confusion matrix
        tn, fp, fn, tp = cm.ravel()
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
        
        print(f"\nMetricas detalladas - {best_model_data['name']}:")
        print(f"True Positives: {tp}")
        print(f"True Negatives: {tn}")
        print(f"False Positives: {fp}")
        print(f"False Negatives: {fn}")
        print(f"Precision: {precision:.4f}")
        print(f"Recall: {recall:.4f}")
        print(f"Specificity: {specificity:.4f}")
        
        # Reporte de clasificacion
        print(f"\nReporte de clasificacion completo:")
        print(classification_report(y_test, y_pred_best, target_names=['Fake', 'Real']))
else:
    print("No hay datos de mejor modelo para matriz de confusion")


Metricas detalladas - stacking:
True Positives: 96
True Negatives: 80
False Positives: 16
False Negatives: 20
Precision: 0.8571
Recall: 0.8276
Specificity: 0.8333

Reporte de clasificacion completo:
              precision    recall  f1-score   support

        Fake       0.80      0.83      0.82        96
        Real       0.86      0.83      0.84       116

    accuracy                           0.83       212
   macro avg       0.83      0.83      0.83       212
weighted avg       0.83      0.83      0.83       212



## Resumen Final y Guardado

Genero el resumen final completo y guardo todos los resultados del sistema de ensemble.

In [16]:
# Generar resumen final completo
def generate_final_summary():
    """Genera resumen ejecutivo final"""
    
    print("ENSEMBLE AVANZADO - RESUMEN FINAL")
    print("-" * 50)
    
    print(f"Timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"Dataset utilizado: Datos procesados del proyecto")
    
    if all_features:
        print(f"\n1. CONFIGURACION EXPERIMENTAL:")
        print(f"   Dataset: {df_clean.shape} filas x columnas")
        print(f"   Features numericas: {len(all_features['numeric_columns'])}")
        if 'text' in all_features:
            print(f"   Features texto TF-IDF: {all_features['text'].shape[1]}")
            print(f"   Total features: {X_train.shape[1]}")
        print(f"   Entrenamiento: {X_train.shape[0]} muestras")
        print(f"   Prueba: {X_test.shape[0]} muestras")
    
    if results_df is not None:
        print(f"\n2. ALGORITMOS INDIVIDUALES:")
        print(f"   Total evaluados: {len(results_df)}")
        print(f"   Mejor individual: {results_df.iloc[0]['name']} F1: {results_df.iloc[0]['f1_score']:.4f}")
        
        print(f"   Top 5:")
        for i, (_, row) in enumerate(results_df.head(5).iterrows(), 1):
            print(f"     {i}. {row['name']}: {row['f1_score']:.4f}")
    
    if ensemble_df is not None:
        print(f"\n3. ENSEMBLES:")
        print(f"   Total construidos: {len(ensemble_df)}")
        print(f"   Mejor ensemble: {ensemble_df.iloc[0]['name']} F1: {ensemble_df.iloc[0]['f1_score']:.4f}")
        
        for _, row in ensemble_df.iterrows():
            print(f"   {row['name']}: F1={row['f1_score']:.4f}")
    
    if comparison_results:
        print(f"\n4. COMPARACION:")
        print(f"   Individual: {comparison_results['best_individual']['name']} F1: {comparison_results['best_individual']['f1_score']:.4f}")
        print(f"   Ensemble: {comparison_results['best_ensemble']['name']} F1: {comparison_results['best_ensemble']['f1_score']:.4f}")
        print(f"   Mejora: {comparison_results['f1_improvement']:+.4f} ({comparison_results['f1_improvement_pct']:+.2f}%)")
        print(f"   Resultado: {comparison_results['verdict']}")
    
    print(f"\n5. RECOMENDACIONES:")
    if comparison_results and comparison_results['verdict'] == 'ENSEMBLE SUPERIOR':
        print(f"   Usar {comparison_results['best_ensemble']['name']} para implementacion")
    
    print("Sistema de ensemble avanzado completado")

# Generar resumen
generate_final_summary()

# Guardar resultados
try:
    results_dir = '../models/ensemble_all_approaches' if not IN_COLAB else 'ensemble_results'
    os.makedirs(results_dir, exist_ok=True)
    
    if results_df is not None:
        results_df.to_csv(f'{results_dir}/individual_results.csv', index=False)
        print(f"Resultados guardados en: {results_dir}")
    
    if ensemble_df is not None:
        ensemble_df.to_csv(f'{results_dir}/ensemble_results.csv', index=False)
    
    if feature_extractor:
        with open(f'{results_dir}/feature_extractor.pkl', 'wb') as f:
            pickle.dump(feature_extractor, f)
    
except Exception as e:
    print(f"Error guardando: {e}")

print("Proceso finalizado")

ENSEMBLE AVANZADO - RESUMEN FINAL
--------------------------------------------------
Timestamp: 2025-08-24 21:01:03
Dataset utilizado: Datos procesados del proyecto

1. CONFIGURACION EXPERIMENTAL:
   Dataset: (1058, 59) filas x columnas
   Features numericas: 57
   Features texto TF-IDF: 1791
   Total features: 1848
   Entrenamiento: 846 muestras
   Prueba: 212 muestras

2. ALGORITMOS INDIVIDUALES:
   Total evaluados: 12
   Mejor individual: ridge_cv F1: 0.8115
   Top 5:
     1. ridge_cv: 0.8115
     2. ridge_basic: 0.8115
     3. svm_linear_l1: 0.8068
     4. catboost_optimized: 0.7947
     5. extra_trees_best: 0.7919

3. ENSEMBLES:
   Total construidos: 3
   Mejor ensemble: stacking F1: 0.8304
   stacking: F1=0.8304
   voting: F1=0.8296
   bagging: F1=0.8017

4. COMPARACION:
   Individual: ridge_cv F1: 0.8115
   Ensemble: stacking F1: 0.8304
   Mejora: +0.0190 (+2.34%)
   Resultado: ENSEMBLE SUPERIOR

5. RECOMENDACIONES:
   Usar stacking para implementacion
Sistema de ensemble avanza